<a href="https://colab.research.google.com/github/vorad1/CS703_dvor451/blob/working/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

from huggingface_hub import login
import os

# Get the secret
token = HF_TOKEN
login(token=token)

In [ ]:
!pip install --upgrade datasets

from datasets import load_dataset

# Force re-download by setting download_mode and clearing cache dir
dataset = load_dataset(
    "commonsense_qa",
    download_mode="force_redownload",
    cache_dir="/content/temp_cache"
)

!pip install -U datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Set up pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Load dataset with custom cache directory
# dataset = load_dataset("commonsense_qa", cache_dir="/content/hf_cache")

# Selecting 2 in-context examples and 1 test question
prompt = """Question: Why would someone wear gloves?
A) To keep their hands clean
B) To make their hands soft
C) To show off
D) To cover scars
Answer: A

Question: Why do people use umbrellas?
A) To fly
B) To block the sun or rain
C) To walk faster
D) To play with
Answer: B

Question: Why do people wear sunglasses?
A) To look cool
B) To protect their eyes from bright light
C) To see better in the dark
D) To avoid being recognized
Answer:"""

# Generate prediction based on training
result = pipe(prompt, max_new_tokens=5)[0]['generated_text']
print("Model's answer:", result)


In [ ]:
!pip install -U datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

# Loading FLAN-T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Set up inference pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

#load dataset with a smaller split for testing
dataset = load_dataset("commonsense_qa", split="validation[:50]")


import random

# using 2 questions from the training set
context_examples = [
  {
    "question": "Why would someone wear gloves?",
    "choices": ["To keep their hands clean", "To make their hands soft", "To show off", "To cover scars"],
    "answer": "A"
  },
  {
    "question": "Why do people use umbrellas?",
    "choices": ["To fly", "To block the sun or rain", "To walk faster", "To play with"],
    "answer": "B"
  }
]

# mapping the letter to create a functions of each letter
letter_map = ["A","B","C","D","E"]

# defining variables to keep track of the score
correct = 0
total = 0

for sample in dataset:
    question = sample["question"]
    choices = sample["choices"]["text"]
    correct_answer_idx = sample["answerKey"]
    correct_answer = correct_answer_idx

    #Building the in-context prompt
    prompt = ""
    for ex in context_examples:
        ex_q = ex["question"]
        ex_choices = ex["choices"]
        ex_ans = ex["answer"]
        prompt += f"Question: {ex_q}\n"
        for i, choice in enumerate(ex_choices):
            prompt += f"{letter_map[i]}) {choice}\n"
        prompt += f"Answer: {ex_ans}\n\n"


    prompt += f"Question: {question}\n"
    for i, choice in enumerate(choices):
        prompt += f"{letter_map[i]}) {choice}\n"
    prompt += "Answer:"

    # generating the answer
    result = pipe(prompt, max_new_tokens=5, do_sample=False)[0]['generated_text'].strip()
    prediction = result[0] if result and result[0] in letter_map else "?"

    # comparing with the truth
    is_correct = (prediction == correct_answer)
    correct += int(is_correct)
    total += 1

    print(f"Q: {question}")
    print(f"Predicted: {prediction}, Actual: {correct_answer}, Correct: {is_correct}")
    print("="*50)

accuracy = correct / total
print(f"\nFinal Accuracy over {total} examples: {accuracy:.2f}")

In [ ]:
!pip install -U datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

# Loading FLAN-T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Set up inference pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

#load dataset with a smaller split for testing
dataset = load_dataset("commonsense_qa", split="validation[:50]")


import random

# using 2 questions from the training set
context_examples = [
  {
    "question": "Why would someone wear gloves?",
    "choices": ["To keep their hands clean", "To make their hands soft", "To show off", "To cover scars"],
    "answer": "A"
  },
  {
    "question": "Why do people use umbrellas?",
    "choices": ["To fly", "To block the sun or rain", "To walk faster", "To play with"],
    "answer": "B"
  }
]

# mapping the letter to create a functions of each letter
letter_map = ["A","B","C","D","E"]

# defining variables to keep track of the score
correct = 0
total = 0

for sample in dataset:
    question = sample["question"]
    choices = sample["choices"]["text"]
    correct_answer_idx = sample["answerKey"]
    correct_answer = correct_answer_idx

    #Building the in-context prompt
    prompt = ""
    for ex in context_examples:
        ex_q = ex["question"]
        ex_choices = ex["choices"]
        ex_ans = ex["answer"]
        prompt += f"Question: {ex_q}\n"
        for i, choice in enumerate(ex_choices):
            prompt += f"{letter_map[i]}) {choice}\n"
        prompt += f"Answer: {ex_ans}\n\n"


    prompt += f"Question: {question}\n"
    for i, choice in enumerate(choices):
        prompt += f"{letter_map[i]}) {choice}\n"
    prompt += "Answer:"

    # generating the answer
    result = pipe(prompt, max_new_tokens=5, do_sample=False)[0]['generated_text'].strip()
    prediction = result[0] if result and result[0] in letter_map else "?"

    # comparing with the truth
    is_correct = (prediction == correct_answer)
    correct += int(is_correct)
    total += 1

    print(f"Q: {question}")
    print(f"Predicted: {prediction}, Actual: {correct_answer}, Correct: {is_correct}")
    print("="*50)

accuracy = correct / total
print(f"\nFinal Accuracy over {total} examples: {accuracy:.2f}")

In [ ]:
!pip install -U datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

# Loading FLAN-T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Set up inference pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

#load dataset with a smaller split for testing
dataset = load_dataset("commonsense_qa", split="validation[:50]")

import csv
import os

# Create a folder to store results (optional)
os.makedirs("results", exist_ok=True)

# **Define `context_examples` and `letter_map` here, before they are used:**
context_examples = [
  {
    "question": "Why would someone wear gloves?",
    "choices": ["To keep their hands clean", "To make their hands soft", "To show off", "To cover scars"],
    "answer": "A"
  },
  {
    "question": "Why do people use umbrellas?",
    "choices": ["To fly", "To block the sun or rain", "To walk faster", "To play with"],
    "answer": "B"
  }
]

letter_map = ["A","B","C","D","E"]
!pip install -U datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset

# Loading FLAN-T5 model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Set up inference pipeline
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

#load dataset with a smaller split for testing
dataset = load_dataset("commonsense_qa", split="validation[:50]")

import csv
import os

# Create a folder to store results (optional)
os.makedirs("results", exist_ok=True)

# **Define `context_examples` and `letter_map` here, before they are used:**
context_examples = [
  {
    "question": "Why would someone wear gloves?",
    "choices": ["To keep their hands clean", "To make their hands soft", "To show off", "To cover scars"],
    "answer": "A"
  },
  {
    "question": "Why do people use umbrellas?",
    "choices": ["To fly", "To block the sun or rain", "To walk faster", "To play with"],
    "answer": "B"
  }
]

letter_map = ["A","B","C","D","E"]

# Open a CSV file to save results
with open("C:\\Users\\chris\\Downloads\\results\\icl_results.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Question", "Choices", "GroundTruth", "Prediction", "Correct"])

    correct = 0
    total = 0

    for sample in dataset:
        question = sample["question"]
        choices = sample["choices"]["text"]
        correct_answer_idx = sample["answerKey"]
        correct_answer = correct_answer_idx

        # Build in-context prompt
        prompt = ""
        for ex in context_examples:
            ex_q = ex["question"]
            ex_choices = ex["choices"]
            ex_ans = ex["answer"]
            prompt += f"Question: {ex_q}\n"
            for i, c in enumerate(ex_choices):
                prompt += f"{letter_map[i]}) {c}\n"
            prompt += f"Answer: {ex_ans}\n\n"

        prompt += f"Question: {question}\n"
        for i, c in enumerate(choices):
            prompt += f"{letter_map[i]}) {c}\n"
        prompt += "Answer:"

        # Generate answer
        result = pipe(prompt, max_new_tokens=5, do_sample=False)[0]['generated_text'].strip()
        prediction = result[0] if result and result[0] in letter_map else "?"

        is_correct = (prediction == correct_answer)
        correct += int(is_correct)
        total += 1

        # Write row to CSV
        writer.writerow([
            question,
            " | ".join(choices),
            correct_answer,
            prediction,
            "Yes" if is_correct else "No"
        ])

        print(f"Processed Q{total}: {prediction} vs. {correct_answer} — {'Correct' if is_correct else 'Wrong'}")

In [ ]:
!pip install -q transformers datasets

from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import random
import pandas as pd
from tqdm import tqdm


# Load the model and tokenizer
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Load and sample CommonsenseQA dataset
dataset = load_dataset("commonsense_qa", split="validation[:100]")

# Define the prompt styles
def format_zero_shot(question, _=None):
    return f"Answer the question: {question}"

def format_few_shot(question, few_examples):
    prompt = ""
    for ex in few_examples:
        prompt += f"Q: {ex['question']}\nA: {ex['answerKey']}\n\n"
    prompt += f"Q: {question}\nA:"
    return prompt

def format_natural_prompt(question, few_examples):
    prompt = "Here are some commonsense questions with answers:\n"
    for ex in few_examples:
        prompt += f"Someone asked: {ex['question']}\nThe answer is: {ex['answerKey']}\n"
    prompt += f"Now answer this: {question}\nAnswer:"
    return prompt

# Choose few-shot examples
few_shot_examples = random.sample(list(dataset), 5)

# Define configurations
configs = {
    "zero-shot": {"shot": 0, "format_func": format_zero_shot},
    "3-shot": {"shot": 3, "format_func": lambda q, exs: format_few_shot(q, exs[:3])},
    "5-shot": {"shot": 5, "format_func": lambda q, exs: format_few_shot(q, exs)},
    "naturalistic": {"shot": 3, "format_func": lambda q, exs: format_natural_prompt(q, exs[:3])}
}

results = []

for config_name, config in configs.items():
    correct = 0
    print(f"\nRunning config: {config_name}")
    for sample in tqdm(dataset):
        prompt = config["format_func"](sample["question"], few_shot_examples)
        output = pipe(prompt, max_new_tokens=10)[0]["generated_text"].strip()
        if sample["answerKey"].lower() in output.lower():
            correct += 1
    accuracy = correct / len(dataset) * 100
    results.append({"Configuration": config_name, "Accuracy (%)": round(accuracy, 2)})


# Save results to CSV
df_results = pd.DataFrame(results)
df_results.to_csv("accuracy_results.csv", index=False)

# Display results
df_results

